In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))  # sobe um nível a partir da pasta models/

In [2]:
import ast
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

from copy import deepcopy
from sklearn.model_selection import KFold

from features_selection.Wrapper import ForwardFeatureSelector

In [3]:
class MLP_100_100_100_classifier(nn.Module):
    def __init__(self, input_len):
        super().__init__()

        self.layer1 = nn.Linear(input_len, 100)  
        self.relu = nn.ReLU()                          
        self.hidden_layer_relu = nn.Sequential(
            nn.Linear(100,100),
            nn.ReLU(),
            nn.Linear(100,100),
            nn.ReLU(),
            nn.Linear(100,100),
            nn.ReLU()
        )
        self.layer2 = nn.Linear(100, 30)

    def forward(self, x):

        x = self.relu(self.layer1(x))
        x = self.hidden_layer_relu(x)
        logits = self.layer2(x)
        
        return logits

In [4]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

y_best_ansatz = np.argmax(y, axis=1)  

# criando o kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [6]:
# implementação de um critério de parada para o modelo parar no ponto "ótimo"
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [7]:
# 3. Função para treinar um fold
def train_fold(model, train_loader, val_loader, criterion, optimizer, n_epochs=1000000, patience = 10, min_delta = 0.0):
    model.to(device)
    early_stopping = EarlyStopping(patience=patience, min_delta=min_delta)
    best_model = None
    best_acc = 0.0
    
    for epoch in range(n_epochs):
        # Treinamento
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.squeeze()  # Converte (batch_size, 1) para (batch_size,)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validação
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            correct = 0
            total = 0
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                y_val = y_val.squeeze()  # Garante que y_val é 1D
                
                outputs = model(X_val)
                val_loss += criterion(outputs, y_val).item()  
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == y_val).sum().item()
                total += y_val.size(0)
        
        
        val_acc = correct / total
        val_loss /= len(val_loader)
        train_loss /= len(train_loader)
        
        print(f'Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.4f}')
        
        # Early Stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
        # Salvar melhor modelo
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = deepcopy(model.state_dict())
    
    return best_model, best_acc

In [8]:
results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n=== Fold {fold + 1}/{3} ===")
    
    # Dividir e preparar dados
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_best_ansatz[train_idx], y_best_ansatz[val_idx]
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    
    # Inicializar modelo e otimizador
    model = MLP_100_100_100_classifier(22)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01) 
    
    # Treinar fold
    best_model, best_acc = train_fold(model, train_loader, val_loader, criterion, optimizer, patience=15)
    results.append(best_acc)
    
    # Salvar modelo se necessário
    torch.save(best_model, f'./models_salvos/best_model_MLP-100-100-100-classifier_fold{fold+1}.pt')

# 7. Resultados finais
print("\n=== Resultados ===")
print(f"Acurácia média: {np.mean(results):.4f} ± {np.std(results):.4f}")
print(f"Acurácias por fold: {results}")


=== Fold 1/3 ===
Epoch 1/1000000 - Train Loss: 3.0600 - Val Loss: 2.9693 - Val Acc: 0.0583
Epoch 2/1000000 - Train Loss: 2.6805 - Val Loss: 2.7137 - Val Acc: 0.3000
Epoch 3/1000000 - Train Loss: 2.3832 - Val Loss: 2.7403 - Val Acc: 0.3000
Epoch 4/1000000 - Train Loss: 2.3475 - Val Loss: 2.7047 - Val Acc: 0.2583
Epoch 5/1000000 - Train Loss: 2.4189 - Val Loss: 2.6940 - Val Acc: 0.2583
Epoch 6/1000000 - Train Loss: 2.2646 - Val Loss: 2.7836 - Val Acc: 0.2583
Epoch 7/1000000 - Train Loss: 2.2470 - Val Loss: 2.7139 - Val Acc: 0.2583
Epoch 8/1000000 - Train Loss: 2.2208 - Val Loss: 2.7727 - Val Acc: 0.2583
Epoch 9/1000000 - Train Loss: 2.2754 - Val Loss: 2.7023 - Val Acc: 0.2583
Epoch 10/1000000 - Train Loss: 2.2262 - Val Loss: 2.6802 - Val Acc: 0.2583
Epoch 11/1000000 - Train Loss: 2.1971 - Val Loss: 2.8158 - Val Acc: 0.2750
Epoch 12/1000000 - Train Loss: 2.2583 - Val Loss: 2.7361 - Val Acc: 0.3000
Epoch 13/1000000 - Train Loss: 2.2085 - Val Loss: 2.7635 - Val Acc: 0.3000
Epoch 14/1000000

In [9]:
model = MLP_100_100_100_classifier(22).to(device)
#carregando o melhor fold treinado
model.load_state_dict(torch.load('.//models_salvos/best_model_MLP-100-100-100-classifier_fold2.pt'))  # Substitua pelo caminho correto
model.eval()  # Modo de avaliação

MLP_100_100_100_classifier(
  (layer1): Linear(in_features=22, out_features=100, bias=True)
  (relu): ReLU()
  (hidden_layer_relu): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): ReLU()
  )
  (layer2): Linear(in_features=100, out_features=30, bias=True)
)

In [10]:
input = X[6]
input_tensor = torch.FloatTensor(input).to(device)

with torch.no_grad():
    logits = model(input_tensor)
    print(logits)
    print(logits.shape)
    probabilities = torch.softmax(logits, dim=0)  # Shape (1, 30)

tensor([ 3.2843, -1.2054,  2.0310, -3.2504, -7.2523, -7.7058,  1.6207,  3.0067,
        -2.1589, -5.3207, -2.5528,  1.1714,  2.4722,  2.0085,  1.5541, -7.7187,
        -1.4011, -3.3925, -4.2345, -3.4395, -2.2506,  1.8444, -2.8576, -4.2709,
        -3.3690, -0.7751, -4.1557, -3.1063, -4.6117, -2.7389])
torch.Size([30])


In [11]:
print(probabilities)

tensor([2.8141e-01, 3.1588e-03, 8.0362e-02, 4.0868e-04, 7.4705e-06, 4.7471e-06,
        5.3318e-02, 2.1319e-01, 1.2173e-03, 5.1552e-05, 8.2098e-04, 3.4020e-02,
        1.2493e-01, 7.8576e-02, 4.9880e-02, 4.6859e-06, 2.5971e-03, 3.5452e-04,
        1.5275e-04, 3.3826e-04, 1.1106e-03, 6.6679e-02, 6.0530e-04, 1.4729e-04,
        3.6297e-04, 4.8573e-03, 1.6528e-04, 4.7199e-04, 1.0476e-04, 6.8157e-04])


In [12]:
# Obter índices e valores das top-k probabilidades (ex.: top-5)
k = 3
top_k_probs, top_k_indices = torch.topk(probabilities, k=k, dim=0)

# Converter para numpy e exibir
top_k_probs = top_k_probs.cpu().numpy().flatten()
top_k_indices = top_k_indices.cpu().numpy().flatten()

print("Melhores ansatzes e suas probabilidades:")
for i, (idx, prob) in enumerate(zip(top_k_indices, top_k_probs)):
    print(f"{i+1}º: Ansatz {idx} - Probabilidade: {prob:.4f}")

Melhores ansatzes e suas probabilidades:
1º: Ansatz 0 - Probabilidade: 0.2814
2º: Ansatz 7 - Probabilidade: 0.2132
3º: Ansatz 12 - Probabilidade: 0.1249


In [13]:
from sklearn.metrics import accuracy_score

selector = ForwardFeatureSelector(
    model=lambda input_dim: MLP_100_100_100_classifier(input_dim),
    model_type='pytorch',
    scoring=accuracy_score,
    cv=3,
    verbose=1,
    pytorch_optimizer= optim.Adam,
)

In [14]:
X_new = selector.fit_transform(X, y_best_ansatz)

Testing feature set: [0], score: 0.3194444444444444
Testing feature set: [1], score: 0.3055555555555555
Testing feature set: [2], score: 0.3055555555555555
Testing feature set: [3], score: 0.3277777777777778
Testing feature set: [4], score: 0.34166666666666673
Testing feature set: [5], score: 0.3194444444444445
Testing feature set: [6], score: 0.3194444444444445
Testing feature set: [7], score: 0.3
Testing feature set: [8], score: 0.3277777777777778
Testing feature set: [9], score: 0.32222222222222224
Testing feature set: [10], score: 0.325
Testing feature set: [11], score: 0.3194444444444445
Testing feature set: [12], score: 0.32222222222222224
Testing feature set: [13], score: 0.3444444444444444
Testing feature set: [14], score: 0.3055555555555556
Testing feature set: [15], score: 0.30833333333333335
Testing feature set: [16], score: 0.27499999999999997
Testing feature set: [17], score: 0.2916666666666667
Testing feature set: [18], score: 0.2916666666666667
Testing feature set: [19],